# **Image Augmentation**

It is a technique used to increase the image with some modification. To overcome overfitting we are using augmentation.

In [1]:
!pip install tensorflow

In [2]:
# Import required lib

from keras.preprocessing.image import ImageDataGenerator

In [3]:
# Initializing augmentation for training variable

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

In [4]:
# Initializing augmentation for testing variable

test_datagen = ImageDataGenerator(rescale=1)

In [ ]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
metadata_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-1c0c3999-6665-44bd-b418-5ea81e4c77c6',
    'IBM_API_KEY_ID': 'JWtMABdtqVklGO22EIJn9nFawFKJ2dbHMRH4L9vRpbS3',
    'ENDPOINT': 'https://s3.private.us.cloud-object-storage.appdomain.cloud',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'fertilizerrecommendationsystemfor-donotdelete-pr-yz7ro2fws2b4p3',
    'FILE': 'Fruit Dataset.zip'
}


In [16]:
from botocore.client import Config
import ibm_boto3

cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='JWtMABdtqVklGO22EIJn9nFawFKJ2dbHMRH4L9vRpbS3',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth',connect_timeout=50, read_timeout=70),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

cos.download_file(Bucket='fertilizerrecommendationsystemfor-donotdelete-pr-yz7ro2fws2b4p3', Key='Fruit Dataset.zip', Filename='Fruit Dataset.zip')

In [17]:
from zipfile import ZipFile

with ZipFile ('Fruit Dataset.zip', 'r') as zipObj:
    zipObj.extractall()

In [23]:
# import os

# os.listdir("./Dataset Plant Disease/fruit-dataset/fruit-dataset/train")

['Apple___healthy',
 'Corn_(maize)___healthy',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Peach___healthy',
 'Peach___Bacterial_spot',
 'Apple___Black_rot']

In [24]:
# Passing training data for training variable (augmentation)

xtrain_fruit = train_datagen.flow_from_directory(r'./Dataset Plant Disease/fruit-dataset/fruit-dataset/train',
                                           target_size=(128,128),
                                           batch_size=32,class_mode='categorical')

Found 5384 images belonging to 6 classes.


In [25]:
# Passing testing data for testing variable (augmentation)

xtest_fruit = test_datagen.flow_from_directory(r'./Dataset Plant Disease/fruit-dataset/fruit-dataset/test',
                                           target_size=(128,128),
                                           batch_size=32,class_mode='categorical')

Found 1686 images belonging to 6 classes.


# **Create Model**

# CNN Model

In [26]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

# **Add Layers (Convolution,MaxPooling,Flatten,Dense-(Hidden Layers),Output)**

In [27]:
# CNN block 

model = Sequential() # Initializing sequential model 
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(128,128,3))) # Convolution layer 
model.add(MaxPooling2D(pool_size=(2, 2))) # Max pooling layer 
model.add(Flatten()) # Flatten layer 

In [28]:
model.add(Dense(40,activation='relu')) # Hidden layer 1
model.add(Dense(20,activation='relu')) # Hidden layer 2
model.add(Dense(6,activation='softmax')) # Output layer 

# **Compile The Model**

In [29]:
# Compile model

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# **Fit The Model**

In [30]:
# Training model

model.fit_generator(xtrain_fruit,
                    steps_per_epoch=len(xtrain_fruit),
                    epochs=50,
                    validation_data=xtest_fruit,
                    validation_steps=len(xtest_fruit))

/tmp/wsuser/ipykernel_164/3744506671.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(xtrain_fruit,


Epoch 1/50
169/169 [==============================] - 88s 519ms/step - loss: 0.8439 - accuracy: 0.7604 - val_loss: 37.6521 - val_accuracy: 0.8701
Epoch 2/50
169/169 [==============================] - 86s 510ms/step - loss: 0.2954 - accuracy: 0.8991 - val_loss: 89.1297 - val_accuracy: 0.7651
Epoch 3/50
169/169 [==============================] - 86s 505ms/step - loss: 0.2366 - accuracy: 0.9190 - val_loss: 108.4242 - val_accuracy: 0.7503
Epoch 4/50
169/169 [==============================] - 85s 503ms/step - loss: 0.2169 - accuracy: 0.9227 - val_loss: 128.6462 - val_accuracy: 0.7479
Epoch 5/50
169/169 [==============================] - 86s 508ms/step - loss: 0.1586 - accuracy: 0.9482 - val_loss: 254.6448 - val_accuracy: 0.6447
Epoch 6/50
169/169 [==============================] - 86s 506ms/step - loss: 0.1620 - accuracy: 0.9434 - val_loss: 180.2370 - val_accuracy: 0.7278
Epoch 7/50
169/169 [==============================] - 89s 525ms/step - loss: 0.1502 - accuracy: 0.9482 - val_loss: 415.1

# **Save The Model**

In [118]:
model_result_path = "fruit_model.h5"
model.save(model_result_path)

In [119]:
!tar -zcvf fruit_model.tgz fruit_model.h5

fruit_model.h5


# **Test The Model**

In [120]:
import os

os.listdir("./Dataset Plant Disease/fruit-dataset/fruit-dataset/")

['train', 'test']

In [38]:
from tensorflow.keras.preprocessing import image  # Importing req. lib
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np

model = load_model("fruit_model.h5")

img = image.load_img(r"./Dataset Plant Disease/fruit-dataset/fruit-dataset/train/Apple___healthy/0055dd26-23a7-4415-ac61-e0b44ebfaf80___RS_HL 5672.JPG",target_size=(128,128))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
pred = np.argmax(model.predict(x))
print(pred)

1


In [58]:
!pip install --upgrade watson-machine-learning-client-V4

     |████████████████████████████████| 1.3 MB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 51 kB 912 kB/s  eta 0:00:01
     |████████████████████████████████| 5.0 MB 47.9 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 824 kB 66.7 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 74.5 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 61.5 MB/s eta 0:00:01
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.7.0-py2.py3-none-any.whl size=72565 sha256=16da46d4abf29d8462fb53ba28204157dfb0cb12cf0e55ab8193ca21c29c2459
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/1d/b5/21/f03b3bea83cdb09171e26f0d928efeab508dba4a05e65a9ae2
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.7.0-py2.py3-none-any.whl size=501014 sha256=6ad7a2ca4ed098391d377d34f490ebd45fa208ccd04daf0a3135b3ea00f08614
  Stored 

In [63]:
# from ibm_watson_machine_learning import APIClient

# wml_credentials = {
#                     "url":"https://us-south.ml.cloud.ibm.com",
#                     "apikey":"_6Vrqs5_bsVKms1b6wlT389sAA5vH5ROR54gnOVznPMz"
#                     }

# client = APIClient(wml_credentials)

wml_credentials = {
                  "apikey":"_6Vrqs5_bsVKms1b6wlT389sAA5vH5ROR54gnOVznPMz",
                  "url": "https://us-south.ml.cloud.ibm.com"
}
from ibm_watson_machine_learning import APIClient
wml_client = APIClient(wml_credentials)


In [79]:
metadata = {            
     client.spaces.ConfigurationMetaNames.NAME: 'FertilizerSpace',         
     client.spaces.ConfigurationMetaNames.DESCRIPTION:  'description',            
     client.spaces.ConfigurationMetaNames.STORAGE: {
             "type": "cloud-object-storage",
             "resource_crn": 'crn:v1:bluemix:public:cloud-object-storage:global:a/ef818577d16748b59dcf44cfce64828a:1497e4c8-ef5b-4e0e-b723-61e0b603ae4d::'
         },
         client.spaces.ConfigurationMetaNames.COMPUTE: {
                      "name": 'pm-20',
                      "crn": 'crn:v1:bluemix:public:pm-20:us-south:a/ef818577d16748b59dcf44cfce64828a:f00401f1-1525-4d43-95e0-cf348c6be93a::' 
         }
     }

space_details = client.spaces.store(meta_props=metadata)

Space has been created. However some background setup activities might still be on-going. Check for 'status' field in the response. It has to show 'active' before space can be used. If it's not 'active', you can monitor the state with a call to spaces.get_details(space_id). Alternatively, use background_mode=False when calling client.spaces.store().


In [84]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    instance_details = client.service_instance.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [85]:
space_uid = guid_from_space_name(client, 'FertilizerSpace')
print("Space UID = " + space_uid)


Space UID = e6246532-fb6f-4bfc-aab4-c4a82516984f


In [86]:
client.set.default_space(space_uid)

'SUCCESS'

In [87]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [139]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [141]:
model_details = client.repository.store_model(model='fruit_model.tgz', meta_props={
    client.repository.ModelMetaNames.NAME:"Fertilizer_Recommendation_System_for_Disease_Prediction",
    client.repository.ModelMetaNames.TYPE: 'tensorflow_2.7',    
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
}
                                             )
model_id = client.repository.get_model_id(model_details)

In [142]:
model_id

'7fa2f26d-e03d-4afa-9f5e-bac3a05de118'

In [143]:
xtrain_fruit[0]

(array([[[[0.54509807, 0.5254902 , 0.54901963],
          [0.54509807, 0.5254902 , 0.54901963],
          [0.54509807, 0.5254902 , 0.54901963],
          ...,
          [0.50843287, 0.48882502, 0.5045113 ],
          [0.5084673 , 0.4888595 , 0.50454575],
          [0.5085018 , 0.48889396, 0.50458026]],
 
         [[0.53428596, 0.5146781 , 0.5382076 ],
          [0.5342745 , 0.5146667 , 0.5381961 ],
          [0.534263  , 0.5146552 , 0.5381846 ],
          ...,
          [0.5423042 , 0.5226964 , 0.53838265],
          [0.54233104, 0.5227232 , 0.5384095 ],
          [0.5423578 , 0.52274996, 0.53843623]],
 
         [[0.5437786 , 0.52417076, 0.54770017],
          [0.54379004, 0.5241822 , 0.5477116 ],
          [0.54380155, 0.5241937 , 0.5477231 ],
          ...,
          [0.5682004 , 0.54859257, 0.56427884],
          [0.56815827, 0.5485504 , 0.5642367 ],
          [0.5681162 , 0.54850835, 0.5641946 ]],
 
         ...,
 
         [[0.70113784, 0.6932947 , 0.71290255],
          [0.70112

In [147]:
model.predict([[[0.61645854, 0.54194874, 0.557635  ],
          [0.6257725 , 0.5512627 , 0.56694895],
          [0.62695605, 0.55244625, 0.5681325 ]]])

array([0])
